In [1]:
import os

In [2]:
%pwd

'/home/zishan/Documents/projects/Text-Summurizer-NLP-Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/zishan/Documents/projects/Text-Summurizer-NLP-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile

from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size


In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file,
            )
            logger.info(f'{filename} downloaded with folowing info:\n{headers}')
        else:
            logger.info(f'{filename} already exists with size {get_size(Path(self.config.local_data_file))}')
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-02 13:32:27,852]: [INFO]: [textSummarizerLogger]: [✅ YAML file loaded successfully: config/config.yaml]
[2025-09-02 13:32:27,853]: [INFO]: [textSummarizerLogger]: [✅ YAML file loaded successfully: params.yaml]
[2025-09-02 13:32:27,853]: [INFO]: [textSummarizerLogger]: [📁 Created directory at /home/zishan/Documents/projects/Text-Summurizer-NLP-Project/artifacts]
[2025-09-02 13:32:27,854]: [INFO]: [textSummarizerLogger]: [📁 Created directory at /home/zishan/Documents/projects/Text-Summurizer-NLP-Project/artifacts/data_ingestion]
[2025-09-02 13:32:31,445]: [INFO]: [textSummarizerLogger]: [artifacts/data_ingestion/data.zip downloaded with folowing info:
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Fra